In [1]:
%env PYSPARK_PYTHON=/usr/local/bin/python3.12

env: PYSPARK_PYTHON=/usr/local/bin/python3.12


In [2]:
import pandas as pd
import random
import string
from datetime import datetime, timedelta

from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType,
    FloatType, DoubleType, BooleanType, DateType, TimestampType, ArrayType
)


# spark = SparkSession \
#     .builder \
#     .appName("MySparkApp") \
#     .master("spark://gpu-n01.cm.cluster:10000") \
#     .config("spark.driver.memory","32G") \
#     .config("spark.driver.extraClassPath","/opt/spark/sparkRapidsPlugin/rapids-4-spark_2.12-25.04.0.jar") \
#     .config("spark.executor.extraClassPath","/opt/spark/sparkRapidsPlugin/rapids-4-spark_2.12-25.04.0.jar") \
#     .config("spark.executor.resource.gpu.amount",1) \
#     .config("spark.plugins","com.nvidia.spark.SQLPlugin") \
#     .config("spark.rapids.allowMultipleJars","ALWAYS") \
#     .config("spark.rapids.filecache.enabled","true") \
#     .config("spark.rapids.memory.host.spillStorageSize","32G") \
#     .config("spark.rapids.memory.pinnedPool.size","8g") \
#     .config("spark.rapids.shuffle.mode","MULTITHREADED") \
#     .config("spark.rapids.shuffle.multiThreaded.reader.threads",24) \
#     .config("spark.rapids.shuffle.multiThreaded.writer.threads",24) \
#     .config("spark.rapids.sql.allowMultipleJars","ALWAYS") \
#     .config("spark.rapids.sql.batchSizeBytes","2gb") \
#     .config("spark.rapids.sql.concurrentGpuTasks",4) \
#     .config("spark.rapids.sql.enabled", True) \
#     .config("spark.rapids.sql.explain","NONE") \
#     .config("spark.sql.cache.serializer", "com.nvidia.spark.ParquetCachedBatchSerializer") \
#     .config("spark.rapids.sql.format.parquet.reader.type", "COALESCING") \
#     .config("spark.rapids.sql.format.parquet.reader.footer.type", "NATIVE") \
#     .config("spark.shuffle.manager","com.nvidia.spark.rapids.spark355.RapidsShuffleManager") \
#     .config("spark.shuffle.compress", False) \
#     .config("spark.executor.cores",12) \
#     .config("spark.executor.instances",4) \
#     .config("spark.executor.memory","16G") \
#     .config("spark.sql.adaptive.advisoryPartitionSizeInBytes","160mb") \
#     .config("spark.sql.adaptive.coalescePartitions.minPartitionSize","32mb") \
#     .config("spark.sql.files.maxPartitionBytes","2gb") \
#     .config("spark.sql.shuffle.partitions",200) \
#     .config("spark.task.cpus",1) \
#     .config("spark.sql.legacy.parquet.datetimeRebaseModeInWrite","CORRECTED") \
#     .config("spark.sql.legacy.charVarcharAsString",True) \
#     .getOrCreate()

spark = SparkSession \
        .builder \
        .appName("MySparkApp") \
        .master("spark://gpu-n01.cm.cluster:10000") \
        .config("spark.driver.memory","31G") \
        .config("spark.driver.cores",5) \
        .config("spark.driver.extraJavaOptions","-Djava.net.preferIPv4Stack=true") \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.sql.session.timeZone", "UTC") \
        .config("spark.executor.cores",10) \
        .config("spark.executor.memory","121g") \
        .config("spark.sql.adaptive.advisoryPartitionSizeInBytes","160mb") \
        .config("spark.sql.adaptive.coalescePartitions.minPartitionSize","32mb") \
        .config("spark.sql.files.maxPartitionBytes","256m") \
        .config("spark.sql.shuffle.partitions",200) \
        .config("spark.task.cpus",1) \
        .config("spark.sql.legacy.charVarcharAsString",True) \
        .config("spark.shuffle.io.numConnectionsPerPeer",2) \
        .config("spark.shuffle.file.buffer",1024) \
        .config("spark.shuffle.unsafe.file.buffer",1024) \
        .config("spark.shuffle.checksum.enabled",False) \
        .config("spark.rdd.compress", True) \
        .config("spark.io.compress", True) \
        .config("spark.shuffle.service.index.cache.size","512m") \
        .config("spark.locality.wait.process","3s") \
        .config("spark.buffer.size", 1024) \
        .config("spark.shuffle.spill.diskWriteBufferSize", 2 * 1024 * 1024) \
        .config("spark.shuffle.checksum.enabled", "false") \
        .config("spark.reducer.maxSizeInFlight", 128) \
        .config("spark.sql.adaptive.enabled", "true") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/20 10:01:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
table_a = spark.read.parquet('/mnt/weka/skew_test/skew_df')

In [4]:
%run -i create_join_tables.py

Found sheets: ['data_sources', 'joins', 'all_joins', 'table_a', 'table_b', 'table_c', 'table_d', 'table_e']
Tables and approximate row counts:
  table_a: ~100000000000 rows
  table_b: ~14000 rows
  table_c: ~2100000000 rows
  table_d: ~12000000 rows
  table_e: ~33000000 rows
Loaded metadata for table 'table_a'.
Loaded metadata for table 'table_b'.
Loaded metadata for table 'table_c'.
Loaded metadata for table 'table_d'.
Loaded metadata for table 'table_e'.
Schema for table 'table_a': StructType([StructField('col_a_0', StringType(), True), StructField('col_a_1', LongType(), True), StructField('col_a_2', LongType(), True), StructField('col_a_3', IntegerType(), True), StructField('col_a_4', ArrayType(IntegerType(), True), True), StructField('col_a_5', LongType(), True), StructField('col_a_6', LongType(), True), StructField('col_a_7', IntegerType(), True), StructField('col_a_8', ArrayType(IntegerType(), True), True), StructField('col_a_9', IntegerType(), True), StructField('col_a_10', Long

   Distinct combinations for table_a: 14000


   ✓ table_a_forced created (0.1% forced)

2. Processing table_c...
   Distinct combinations for table_c: 14000


   ✓ table_c_forced created (1.0% forced)

3. Processing table_d...
   Distinct combinations for table_d: 14000


   ✓ table_d_forced created (1.0% forced)

4. Processing table_e...
   Distinct values for table_e: 14000


   ✓ table_e_forced created (1.0% forced)


In [7]:
# table_a_forced = spark.read.parquet("/mnt/weka/temp_forced_table_a") # 100 GB, 5B Rows
# table_b = spark.read.parquet("/mnt/weka/temp_forced_table_b") # 14000 Rows
# table_c_forced = spark.read.parquet("/mnt/weka/temp_forced_table_c") # 2100000000 Rows
# table_d_forced = spark.read.parquet("/mnt/weka/temp_forced_table_d") # 12000000 Rows
# table_e_forced = spark.read.parquet("/mnt/weka/temp_forced_table_e") # 33000000 Rows

linked_join_test_forced = (
    table_a_forced
    .join(
        F.broadcast(table_b),
        [
            table_a_forced["col_a"] == table_b["col_b_8"],
            table_a_forced["col_c"] == table_b["col_b_3"],
            table_a_forced["col_b"] == table_b["col_b_9"],
            table_a_forced["col_d"] == table_b["col_b_1"],
        ],
        how="inner"
    )
    .join(
        table_c_forced,
        [
            table_a_forced["col_a"] == table_c_forced["col_c_10"],
            table_a_forced["col_b"] == table_c_forced["col_c_9"],
            table_a_forced["col_c"] == table_c_forced["col_c_11"],
        ],
        how="inner"
    )
    .join(
        table_d_forced,
        [
            table_a_forced["col_a"] == table_d_forced["col_d_0"],
            table_a_forced["col_b"] == table_d_forced["col_d_1"],
        ],
        how="inner"
    )
    .join(
        table_e_forced,
        table_a_forced["col_a"] == table_e_forced["col_e_0"],
        how="inner"
    )
)

In [8]:
table_a_forced.count()

4999992075

In [ ]:
sc = spark.sparkContext
sc.setJobDescription("Forced Linked Join Test")
linked_join_test_forced.write.parquet("/mnt/weka/eddie_workflow_test", mode="overwrite")

[Stage 36:=========>                                            (73 + 40) / 422]

# Spark-Rapids Plan
```
== Physical Plan ==
AdaptiveSparkPlan (96)
+- == Final Plan ==
   GpuColumnarToRow (52)
   +- Execute GpuInsertIntoHadoopFsRelationCommand (51)
      +- GpuWriteFiles (50)
         +- GpuShuffledSymmetricHashJoin (49)
            :- ShuffleQueryStage (43), Statistics(sizeInBytes=287.4 GiB, rowCount=6.45E+8)
            :  +- GpuColumnarExchange (42)
            :     +- GpuShuffledSymmetricHashJoin (41)
            :        :- ShuffleQueryStage (35), Statistics(sizeInBytes=25.8 GiB, rowCount=7.51E+7)
            :        :  +- GpuColumnarExchange (34)
            :        :     +- GpuShuffledSymmetricHashJoin (33)
            :        :        :- GpuCustomShuffleReader (26)
            :        :        :  +- ShuffleQueryStage (25), Statistics(sizeInBytes=1035.9 MiB, rowCount=5.00E+6)
            :        :        :     +- GpuColumnarExchange (24)
            :        :        :        +- GpuBroadcastHashJoin (23)
            :        :        :           :- GpuCoalesceBatches (3)
            :        :        :           :  +- GpuFilter (2)
            :        :        :           :     +- GpuScan parquet  (1)
            :        :        :           +- BroadcastQueryStage (22), Statistics(sizeInBytes=2.1 MiB, rowCount=1.40E+4)
            :        :        :              +- GpuBroadcastExchange (21)
            :        :        :                 +- GpuCoalesceBatches (20)
            :        :        :                    +- GpuFilter (19)
            :        :        :                       +- GpuProject (18)
            :        :        :                          +- GpuProject (17)
            :        :        :                             +- GpuProject (16)
            :        :        :                                +- GpuProject (15)
            :        :        :                                   +- GpuProject (14)
            :        :        :                                      +- GpuProject (13)
            :        :        :                                         +- GpuProject (12)
            :        :        :                                            +- GpuProject (11)
            :        :        :                                               +- GpuProject (10)
            :        :        :                                                  +- GpuProject (9)
            :        :        :                                                     +- GpuProject (8)
            :        :        :                                                        +- GpuProject (7)
            :        :        :                                                           +- GpuProject (6)
            :        :        :                                                              +- GpuProject (5)
            :        :        :                                                                 +- GpuRange (4)
            :        :        +- GpuCustomShuffleReader (32)
            :        :           +- ShuffleQueryStage (31), Statistics(sizeInBytes=3.1 GiB, rowCount=2.10E+7)
            :        :              +- GpuColumnarExchange (30)
            :        :                 +- GpuCoalesceBatches (29)
            :        :                    +- GpuFilter (28)
            :        :                       +- GpuScan parquet  (27)
            :        +- ShuffleQueryStage (40), Statistics(sizeInBytes=1276.4 MiB, rowCount=1.20E+7)
            :           +- GpuColumnarExchange (39)
            :              +- GpuCoalesceBatches (38)
            :                 +- GpuFilter (37)
            :                    +- GpuScan parquet  (36)
            +- ShuffleQueryStage (48), Statistics(sizeInBytes=6.0 GiB, rowCount=3.30E+7)
               +- GpuColumnarExchange (47)
                  +- GpuCoalesceBatches (46)
                     +- GpuFilter (45)
                        +- GpuScan parquet  (44)
+- == Initial Plan ==
   Execute InsertIntoHadoopFsRelationCommand (95)
   +- WriteFiles (94)
      +- SortMergeJoin Inner (93)
         :- Sort (88)
         :  +- Exchange (87)
         :     +- SortMergeJoin Inner (86)
         :        :- Sort (81)
         :        :  +- Exchange (80)
         :        :     +- SortMergeJoin Inner (79)
         :        :        :- Sort (74)
         :        :        :  +- Exchange (73)
         :        :        :     +- BroadcastHashJoin Inner BuildRight (72)
         :        :        :        :- Filter (54)
         :        :        :        :  +- Scan parquet  (53)
         :        :        :        +- BroadcastExchange (71)
         :        :        :           +- Filter (70)
         :        :        :              +- Project (69)
         :        :        :                 +- Project (68)
         :        :        :                    +- Project (67)
         :        :        :                       +- Project (66)
         :        :        :                          +- Project (65)
         :        :        :                             +- Project (64)
         :        :        :                                +- Project (63)
         :        :        :                                   +- Project (62)
         :        :        :                                      +- Project (61)
         :        :        :                                         +- Project (60)
         :        :        :                                            +- Project (59)
         :        :        :                                               +- Project (58)
         :        :        :                                                  +- Project (57)
         :        :        :                                                     +- Project (56)
         :        :        :                                                        +- Range (55)
         :        :        +- Sort (78)
         :        :           +- Exchange (77)
         :        :              +- Filter (76)
         :        :                 +- Scan parquet  (75)
         :        +- Sort (85)
         :           +- Exchange (84)
         :              +- Filter (83)
         :                 +- Scan parquet  (82)
         +- Sort (92)
            +- Exchange (91)
               +- Filter (90)
                  +- Scan parquet  (89)


(1) GpuScan parquet 
Output [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_a]
PushedFilters: [IsNotNull(col_a), IsNotNull(col_c), IsNotNull(col_b), IsNotNull(col_d)]
ReadSchema: struct<col_a:bigint,col_b:bigint,col_c:bigint,col_d:bigint,col_e:string,source:string>

(2) GpuFilter
Input [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Arguments: (((gpuisnotnull(col_a#543L) AND gpuisnotnull(col_c#545L)) AND gpuisnotnull(col_b#544L)) AND gpuisnotnull(col_d#546L)), [loreId=183]

(3) GpuCoalesceBatches
Input [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Arguments: targetsize(2147483648), [loreId=184]

(4) GpuRange
Output [1]: [id#12L]
Arguments: 0, 14000, 1, 24, [id#12L], 2147483648, [loreId=128]

(5) GpuProject
Input [1]: [id#12L]
Arguments: [id#12L, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(-7107608758632108270, false), 42), false) as string)) AS col_b_0#14], true, [loreId=129]

(6) GpuProject
Input [2]: [id#12L, col_b_0#14]
Arguments: [id#12L, col_b_0#14, cast(((gpurand(-3721675188423819449, false) * 65529.0) + 5.0) as int) AS col_b_1#17], true, [loreId=130]

(7) GpuProject
Input [3]: [id#12L, col_b_0#14, col_b_1#17]
Arguments: [id#12L, col_b_0#14, col_b_1#17, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(-7624506662613589358, false), 42), false) as string)) AS col_b_2#21], true, [loreId=131]

(8) GpuProject
Input [4]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, cast(((gpurand(-3420650322515237113, false) * 65532.0) + 2.0) as int) AS col_b_3#26], true, [loreId=132]

(9) GpuProject
Input [5]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, cast(((gpurand(8755738886740153944, false) * 8637610.0) + 32.0) as bigint) AS col_b_4#32L], true, [loreId=133]

(10) GpuProject
Input [6]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(-4992624780321176895, false), 42), false) as string)) AS col_b_5#39], true, [loreId=134]

(11) GpuProject
Input [7]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(-7112768952529686126, false), 42), false) as string)) AS col_b_6#47], true, [loreId=135]

(12) GpuProject
Input [8]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(-6690459020751281288, false), 42), false) as string)) AS col_b_7#56], true, [loreId=136]

(13) GpuProject
Input [9]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, cast(((gpurand(-675358647604847885, false) * 3.740470335E9) + 1.6843009E7) as bigint) AS col_b_8#66L], true, [loreId=137]

(14) GpuProject
Input [10]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, cast(((gpurand(3527641861187337108, false) * 3.708560065E9) + 3.7263149E7) as bigint) AS col_b_9#77L], true, [loreId=138]

(15) GpuProject
Input [11]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, cast(((gpurand(8755511000376541438, false) * 109.0) + 100.0) as int) AS col_b_10#89], true, [loreId=139]

(16) GpuProject
Input [12]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, cast(((gpurand(559774764092467905, false) * 2.69494294E9) + 1.6E9) as bigint) AS col_b_11#102L], true, [loreId=140]

(17) GpuProject
Input [13]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L]
Arguments: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, cast(((gpurand(1858836248743617155, false) * 2.69494294E9) + 1.6E9) as bigint) AS col_b_12#116L], true, [loreId=141]

(18) GpuProject
Input [14]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L]
Arguments: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, gpuconcat(str_, cast(gpuabs(hash(id#12L, gpurand(4011423678934869014, false), 42), false) as string)) AS col_b_13#131], true, [loreId=142]

(19) GpuFilter
Input [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: (((gpuisnotnull(col_b_8#66L) AND gpuisnotnull(col_b_3#26)) AND gpuisnotnull(col_b_9#77L)) AND gpuisnotnull(col_b_1#17)), [loreId=143]

(20) GpuCoalesceBatches
Input [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: targetsize(2147483648), [loreId=144]

(21) GpuBroadcastExchange
Input [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: HashedRelationBroadcastMode(List(input[8, bigint, false], cast(input[3, int, false] as bigint), input[9, bigint, false], cast(input[1, int, false] as bigint)),false), [plan_id=3197], [loreId=145]

(22) BroadcastQueryStage
Output [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: 0

(23) GpuBroadcastHashJoin
Left output [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Right output [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: [col_a#543L, col_c#545L, col_b#544L, col_d#546L], [col_b_8#66L, cast(col_b_3#26 as bigint), col_b_9#77L, cast(col_b_1#17 as bigint)], Inner, GpuBuildRight, [loreId=185]

(24) GpuColumnarExchange
Input [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: gpuhashpartitioning(col_a#543L, col_b#544L, col_c#545L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3486], [loreId=186]

(25) ShuffleQueryStage
Output [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: 4

(26) GpuCustomShuffleReader
Input [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: coalesced

(27) GpuScan parquet 
Output [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_c]
PushedFilters: [IsNotNull(col_c_10), IsNotNull(col_c_9), IsNotNull(col_c_11)]
ReadSchema: struct<col_c_0:string,col_c_1:array<int>,col_c_2:string,col_c_3:array<int>,col_c_4:bigint,col_c_5:bigint,col_c_6:string,col_c_7:bigint,col_c_8:bigint,col_c_9:bigint,col_c_10:bigint,col_c_11:bigint,col_c_12:string>

(28) GpuFilter
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: ((gpuisnotnull(col_c_10#721L) AND gpuisnotnull(col_c_9#720L)) AND gpuisnotnull(col_c_11#722L)), [loreId=150]

(29) GpuCoalesceBatches
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: targetsize(2147483648), [loreId=151]

(30) GpuColumnarExchange
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: gpuhashpartitioning(col_c_10#721L, col_c_9#720L, col_c_11#722L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3251], [loreId=152]

(31) ShuffleQueryStage
Output [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: 1

(32) GpuCustomShuffleReader
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: coalesced

(33) GpuShuffledSymmetricHashJoin
Left output [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Right output [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: Inner, [col_a#543L, col_b#544L, col_c#545L], [col_c_10#721L, col_c_9#720L, col_c_11#722L], false, 2147483648, 1.0, CoalesceReadOption(false,Never,None), false, [loreId=221]

(34) GpuColumnarExchange
Input [33]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: gpuhashpartitioning(col_a#543L, col_b#544L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3889], [loreId=222]

(35) ShuffleQueryStage
Output [33]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: 5

(36) GpuScan parquet 
Output [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_d]
PushedFilters: [IsNotNull(col_d_0), IsNotNull(col_d_1)]
ReadSchema: struct<col_d_0:bigint,col_d_1:bigint,col_d_2:string,col_d_3:string,col_d_4:bigint,col_d_5:string,col_d_6:string,col_d_7:string>

(37) GpuFilter
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: (gpuisnotnull(col_d_0#828L) AND gpuisnotnull(col_d_1#829L)), [loreId=157]

(38) GpuCoalesceBatches
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: targetsize(2147483648), [loreId=158]

(39) GpuColumnarExchange
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: gpuhashpartitioning(col_d_0#828L, col_d_1#829L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3282], [loreId=159]

(40) ShuffleQueryStage
Output [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: 2

(41) GpuShuffledSymmetricHashJoin
Left output [33]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Right output [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: Inner, [col_a#543L, col_b#544L], [col_d_0#828L, col_d_1#829L], false, 2147483648, 1.0, CoalesceReadOption(false,Never,None), false, [loreId=230]

(42) GpuColumnarExchange
Input [41]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: gpuhashpartitioning(col_a#543L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3994], [loreId=231]

(43) ShuffleQueryStage
Output [41]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: 6

(44) GpuScan parquet 
Output [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_e]
PushedFilters: [IsNotNull(col_e_0)]
ReadSchema: struct<col_e_0:bigint,col_e_1:string,col_e_2:string,col_e_3:bigint,col_e_4:string,col_e_5:array<string>,col_e_6:array<string>,col_e_7:array<string>,col_e_8:string>

(45) GpuFilter
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: gpuisnotnull(col_e_0#924L), [loreId=164]

(46) GpuCoalesceBatches
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: targetsize(2147483648), [loreId=165]

(47) GpuColumnarExchange
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: gpuhashpartitioning(col_e_0#924L, 200, Murmur3Mode), ENSURE_REQUIREMENTS, [plan_id=3318], [loreId=166]

(48) ShuffleQueryStage
Output [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: 3

(49) GpuShuffledSymmetricHashJoin
Left output [41]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Right output [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: Inner, [col_a#543L], [col_e_0#924L], false, 2147483648, 1.0, CoalesceReadOption(false,Never,None), false, [loreId=238]

(50) GpuWriteFiles
Input [50]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835, col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]

(51) Execute GpuInsertIntoHadoopFsRelationCommand
Input: []
Arguments: file:/mnt/weka/eddie_workflow_test, false, com.nvidia.spark.rapids.GpuParquetFileFormat@39717a63, [path=/mnt/weka/eddie_workflow_test], Overwrite, [col_a, col_b, col_c, col_d, col_e, source, col_b_0, col_b_1, col_b_2, col_b_3, col_b_4, col_b_5, col_b_6, col_b_7, col_b_8, col_b_9, col_b_10, col_b_11, col_b_12, col_b_13, col_c_0, col_c_1, col_c_2, col_c_3, ... 26 more fields], false, 0

(52) GpuColumnarToRow
Input: []
Arguments: false, [loreId=241]

(53) Scan parquet 
Output [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_a]
PushedFilters: [IsNotNull(col_a), IsNotNull(col_c), IsNotNull(col_b), IsNotNull(col_d)]
ReadSchema: struct<col_a:bigint,col_b:bigint,col_c:bigint,col_d:bigint,col_e:string,source:string>

(54) Filter
Input [6]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548]
Condition : (((isnotnull(col_a#543L) AND isnotnull(col_c#545L)) AND isnotnull(col_b#544L)) AND isnotnull(col_d#546L))

(55) Range
Output [1]: [id#12L]
Arguments: Range (0, 14000, step=1, splits=Some(24))

(56) Project
Output [2]: [id#12L, concat(str_, cast(abs(hash(id#12L, rand(-7107608758632108270), 42)) as string)) AS col_b_0#14]
Input [1]: [id#12L]

(57) Project
Output [3]: [id#12L, col_b_0#14, cast(((rand(-3721675188423819449) * 65529.0) + 5.0) as int) AS col_b_1#17]
Input [2]: [id#12L, col_b_0#14]

(58) Project
Output [4]: [id#12L, col_b_0#14, col_b_1#17, concat(str_, cast(abs(hash(id#12L, rand(-7624506662613589358), 42)) as string)) AS col_b_2#21]
Input [3]: [id#12L, col_b_0#14, col_b_1#17]

(59) Project
Output [5]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, cast(((rand(-3420650322515237113) * 65532.0) + 2.0) as int) AS col_b_3#26]
Input [4]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21]

(60) Project
Output [6]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, cast(((rand(8755738886740153944) * 8637610.0) + 32.0) as bigint) AS col_b_4#32L]
Input [5]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26]

(61) Project
Output [7]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, concat(str_, cast(abs(hash(id#12L, rand(-4992624780321176895), 42)) as string)) AS col_b_5#39]
Input [6]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L]

(62) Project
Output [8]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, concat(str_, cast(abs(hash(id#12L, rand(-7112768952529686126), 42)) as string)) AS col_b_6#47]
Input [7]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39]

(63) Project
Output [9]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, concat(str_, cast(abs(hash(id#12L, rand(-6690459020751281288), 42)) as string)) AS col_b_7#56]
Input [8]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47]

(64) Project
Output [10]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, cast(((rand(-675358647604847885) * 3.740470335E9) + 1.6843009E7) as bigint) AS col_b_8#66L]
Input [9]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56]

(65) Project
Output [11]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, cast(((rand(3527641861187337108) * 3.708560065E9) + 3.7263149E7) as bigint) AS col_b_9#77L]
Input [10]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L]

(66) Project
Output [12]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, cast(((rand(8755511000376541438) * 109.0) + 100.0) as int) AS col_b_10#89]
Input [11]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L]

(67) Project
Output [13]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, cast(((rand(559774764092467905) * 2.69494294E9) + 1.6E9) as bigint) AS col_b_11#102L]
Input [12]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89]

(68) Project
Output [14]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, cast(((rand(1858836248743617155) * 2.69494294E9) + 1.6E9) as bigint) AS col_b_12#116L]
Input [13]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L]

(69) Project
Output [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, concat(str_, cast(abs(hash(id#12L, rand(4011423678934869014), 42)) as string)) AS col_b_13#131]
Input [14]: [id#12L, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L]

(70) Filter
Input [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Condition : (((isnotnull(col_b_8#66L) AND isnotnull(col_b_3#26)) AND isnotnull(col_b_9#77L)) AND isnotnull(col_b_1#17))

(71) BroadcastExchange
Input [14]: [col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: HashedRelationBroadcastMode(List(input[8, bigint, false], cast(input[3, int, false] as bigint), input[9, bigint, false], cast(input[1, int, false] as bigint)),false), [plan_id=2616]

(72) BroadcastHashJoin
Left keys [4]: [col_a#543L, col_c#545L, col_b#544L, col_d#546L]
Right keys [4]: [col_b_8#66L, cast(col_b_3#26 as bigint), col_b_9#77L, cast(col_b_1#17 as bigint)]
Join type: Inner
Join condition: None

(73) Exchange
Input [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: hashpartitioning(col_a#543L, col_b#544L, col_c#545L, 200), ENSURE_REQUIREMENTS, [plan_id=2620]

(74) Sort
Input [20]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131]
Arguments: [col_a#543L ASC NULLS FIRST, col_b#544L ASC NULLS FIRST, col_c#545L ASC NULLS FIRST], false, 0

(75) Scan parquet 
Output [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_c]
PushedFilters: [IsNotNull(col_c_10), IsNotNull(col_c_9), IsNotNull(col_c_11)]
ReadSchema: struct<col_c_0:string,col_c_1:array<int>,col_c_2:string,col_c_3:array<int>,col_c_4:bigint,col_c_5:bigint,col_c_6:string,col_c_7:bigint,col_c_8:bigint,col_c_9:bigint,col_c_10:bigint,col_c_11:bigint,col_c_12:string>

(76) Filter
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Condition : ((isnotnull(col_c_10#721L) AND isnotnull(col_c_9#720L)) AND isnotnull(col_c_11#722L))

(77) Exchange
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: hashpartitioning(col_c_10#721L, col_c_9#720L, col_c_11#722L, 200), ENSURE_REQUIREMENTS, [plan_id=2621]

(78) Sort
Input [13]: [col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: [col_c_10#721L ASC NULLS FIRST, col_c_9#720L ASC NULLS FIRST, col_c_11#722L ASC NULLS FIRST], false, 0

(79) SortMergeJoin
Left keys [3]: [col_a#543L, col_b#544L, col_c#545L]
Right keys [3]: [col_c_10#721L, col_c_9#720L, col_c_11#722L]
Join type: Inner
Join condition: None

(80) Exchange
Input [33]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: hashpartitioning(col_a#543L, col_b#544L, 200), ENSURE_REQUIREMENTS, [plan_id=2627]

(81) Sort
Input [33]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723]
Arguments: [col_a#543L ASC NULLS FIRST, col_b#544L ASC NULLS FIRST], false, 0

(82) Scan parquet 
Output [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_d]
PushedFilters: [IsNotNull(col_d_0), IsNotNull(col_d_1)]
ReadSchema: struct<col_d_0:bigint,col_d_1:bigint,col_d_2:string,col_d_3:string,col_d_4:bigint,col_d_5:string,col_d_6:string,col_d_7:string>

(83) Filter
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Condition : (isnotnull(col_d_0#828L) AND isnotnull(col_d_1#829L))

(84) Exchange
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: hashpartitioning(col_d_0#828L, col_d_1#829L, 200), ENSURE_REQUIREMENTS, [plan_id=2628]

(85) Sort
Input [8]: [col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: [col_d_0#828L ASC NULLS FIRST, col_d_1#829L ASC NULLS FIRST], false, 0

(86) SortMergeJoin
Left keys [2]: [col_a#543L, col_b#544L]
Right keys [2]: [col_d_0#828L, col_d_1#829L]
Join type: Inner
Join condition: None

(87) Exchange
Input [41]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: hashpartitioning(col_a#543L, 200), ENSURE_REQUIREMENTS, [plan_id=2634]

(88) Sort
Input [41]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835]
Arguments: [col_a#543L ASC NULLS FIRST], false, 0

(89) Scan parquet 
Output [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Batched: true
Location: InMemoryFileIndex [file:/mnt/weka/temp_forced_table_e]
PushedFilters: [IsNotNull(col_e_0)]
ReadSchema: struct<col_e_0:bigint,col_e_1:string,col_e_2:string,col_e_3:bigint,col_e_4:string,col_e_5:array<string>,col_e_6:array<string>,col_e_7:array<string>,col_e_8:string>

(90) Filter
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Condition : isnotnull(col_e_0#924L)

(91) Exchange
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: hashpartitioning(col_e_0#924L, 200), ENSURE_REQUIREMENTS, [plan_id=2635]

(92) Sort
Input [9]: [col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]
Arguments: [col_e_0#924L ASC NULLS FIRST], false, 0

(93) SortMergeJoin
Left keys [1]: [col_a#543L]
Right keys [1]: [col_e_0#924L]
Join type: Inner
Join condition: None

(94) WriteFiles
Input [50]: [col_a#543L, col_b#544L, col_c#545L, col_d#546L, col_e#547, source#548, col_b_0#14, col_b_1#17, col_b_2#21, col_b_3#26, col_b_4#32L, col_b_5#39, col_b_6#47, col_b_7#56, col_b_8#66L, col_b_9#77L, col_b_10#89, col_b_11#102L, col_b_12#116L, col_b_13#131, col_c_0#711, col_c_1#712, col_c_2#713, col_c_3#714, col_c_4#715L, col_c_5#716L, col_c_6#717, col_c_7#718L, col_c_8#719L, col_c_9#720L, col_c_10#721L, col_c_11#722L, col_c_12#723, col_d_0#828L, col_d_1#829L, col_d_2#830, col_d_3#831, col_d_4#832L, col_d_5#833, col_d_6#834, col_d_7#835, col_e_0#924L, col_e_1#925, col_e_2#926, col_e_3#927L, col_e_4#928, col_e_5#929, col_e_6#930, col_e_7#931, col_e_8#932]

(95) Execute InsertIntoHadoopFsRelationCommand
Input: []
Arguments: file:/mnt/weka/eddie_workflow_test, false, Parquet, [path=/mnt/weka/eddie_workflow_test], Overwrite, [col_a, col_b, col_c, col_d, col_e, source, col_b_0, col_b_1, col_b_2, col_b_3, col_b_4, col_b_5, col_b_6, col_b_7, col_b_8, col_b_9, col_b_10, col_b_11, col_b_12, col_b_13, col_c_0, col_c_1, col_c_2, col_c_3, ... 26 more fields]

(96) AdaptiveSparkPlan
Output: []
Arguments: isFinalPlan=true

```